packages needed: urllib.request, urllib.parse, requests, pyfits, numpy, scipy.signal

In [1]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
import os
import urllib.request
import urllib.parse
import requests
import pyfits
import numpy
import scipy.signal

class lamost:
    email=None
    token=None
    dataset=None
    version=None
    __isdev=False

    def __init__(self, isdev=False, dataset=5):
        self.__isdev=isdev
        self.dataset=dataset

    def __getDataset(self):
        prefix='dr5'
        if self.dataset is not None:
            prefix = 'dr%d'%self.dataset
        if self.__isdev: return 'l'+prefix
        else: return prefix

    def __getVersion(self):
        if self.version is not None:
            return '/v%d'%self.version
        return ''

    __config=None
    __config_file_path=os.path.expanduser('~')+'/pylamost.ini'
    
    def __getConfig(self, reload=False):
        if not os.path.exists(self.__config_file_path): return None
        
        if not reload and None!=self.__config: return self.__config
        
        with open(self.__config_file_path) as fh:
            self.__config={}
            for line in fh:
                if line.startswith('#'):continue
                k,v=line.split("=")
                self.__config[k.strip()]=v.strip()
        return self.__config

    def __detectToken(self):
        if self.token is not None: return True
        cf = self.__getConfig()
        if cf is None or 'token' not in cf.keys(): 
            print('please set your token')
            return False
        self.token=cf['token']
        return True

    def download(self, url, savedir='./'):
        response = urllib.request.urlopen(url)
        data = response.read()
        savefile=savedir+'/'+response.getheader("Content-disposition").split('=')[1]
        with open(savefile, 'wb') as fh:
            fh.write(data)

    def getUrl(self, url, params=None):
        if params is None:
            response = urllib.request.urlopen(url)
        else:
            response = urllib.request.urlopen(url, urllib.parse.urlencode(params).encode('utf-8'))
        chrset = response.headers.get_content_charset()
        if chrset is None: chrset='utf-8'
        data = response.read().decode(chrset)
        return data

    def downloadCatalog(self, catname, savedir='./'):
        caturl='http://{0}.lamost.org{1}/catdl?name={2}&token={3}'.format(self.__getDataset(), self.__getVersion(), catname, self.token)
        self.download(url, savedir)

    def downloadFits(self, obsid, savedir='./'):
        if not self.__detectToken(): return
        fitsurl='http://{0}.lamost.org{1}/spectrum/fits/{2}?token={3}'.format(self.__getDataset(), self.__getVersion(), obsid, self.token)
        self.download(fitsurl, savedir)

    def downloadPng(self, obsid, savedir='./'):
        if not self.__detectToken(): return
        pngurl='http://{0}.lamost.org{1}/spectrum/png/{2}?token={3}'.format(self.__getDataset(), self.__getVersion(), obsid, self.token)
        self.download(pngurl, savedir)

    def getFitsCsv(self, obsid):
        if not self.__detectToken(): return None
        url='http://{0}.lamost.org{1}/spectrum/fits2csv/{2}?token={3}'.format(self.__getDataset(), self.__getVersion(), obsid, self.token)
        return self.getUrl(url)

    def getInfo(self, obsid):
        if not self.__detectToken(): return None
        #url='http://{0}.lamost.org{1}/spectrum/info/{2}?token={3}'.format(self.__getDataset(), self.__getVersion(), obsid, self.token)
        #return self.getUrl(url, params)
        url='http://{0}.lamost.org{1}/spectrum/info/{2}'.format(self.__getDataset(), self.__getVersion(), obsid)
        return self.getUrl(url, {'token':self.token})

    #Cone Search Protocol
    def conesearch(self, ra, dec, radius):
        if not self.__detectToken(): return
        conesearchurl='http://{0}.lamost.org{1}/voservice/conesearch?ra={2}&dec={3}&sr={4}&token={5}'.format(self.__getDataset(), self.__getVersion(), ra, dec, radius, self.token)
        return self.getUrl(conesearchurl)

    #Simple Spectral Access Protocol
    def ssap(self, ra, dec, radius):
        if not self.__detectToken(): return
        ssapurl='http://{0}.lamost.org{1}/voservice/ssap?pos={2},{3}&size={4}&token={5}'.format(self.__getDataset(), self.__getVersion(), ra, dec, radius, self.token)
        return self.getUrl(ssapurl)

    def sql(self, sql):
        if not self.__detectToken(): return
        sqlurl='http://{0}.lamost.org{1}/sql/q?&token={2}'.format(self.__getDataset(), self.__getVersion(), self.token)
        return self.getUrl(sqlurl, {'output.fmt':'csv', 'sql':sql})

    def query(self, params):
        if not self.__detectToken(): return
        qurl='http://{0}.lamost.org{1}/q?token={2}'.format(self.__getDataset(), self.__getVersion(), self.token)
        return self.getUrl(qurl, params)
    
    def query2(self, params, files):
        if not self.__detectToken(): return
        qurl='http://{0}.lamost.org{1}/q?token={2}'.format(self.__getDataset(), self.__getVersion(),self.token)
        r=requests.post(qurl, data=params, files=files)
        return str(r.text)
    
    def readFits(self, filename):
        hdulist = pyfits.open(filename)
        head = hdulist[0].header
        scidata = hdulist[0].data
        coeff0 = head['COEFF0']
        coeff1 = head['COEFF1']
        pixel_num = head['NAXIS1'] 
        specflux = scidata[0,]
        spec_noconti = scidata[2,]
        wavelength=numpy.linspace(0,pixel_num,pixel_num)
        wavelength=numpy.power(10,(coeff0+wavelength*coeff1))
        hdulist.close()
        #
        spec_smooth_7=scipy.signal.medfilt(specflux,7)
        spec_smooth_15=scipy.signal.medfilt(specflux,15)
        return (wavelength, specflux, spec_smooth_7, spec_smooth_15)
    

C:\Anaconda3-v5.0.1-x64\lib\site-packages\pyfits-3.5-py3.6-win-amd64.egg\pyfits\__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


In [2]:
def showHtml(html):
    from IPython.core.display import display, HTML
    display(HTML(html))

In order to access protected LAMOST data, you should provide your token.
Your token can be obtained from http://dr.lamost.org/ucenter/mytoken, the "Password" is the token.
When you get your token you can also create a file `~/pylamost.ini` or `%userprofile%/pylamost.ini` on Windows, with content:
```
token=12345678
```
Then you don't have to write the token next time.

In [3]:
lm=lamost()#init the lamost class
#lm.token='12345678'#specify your token. You can pass this step, if you created the ~/pylamost.ini file.
lm.dataset=5 #specify the Data Release number
#lm.version=1 #specify the data version, or leave it None to always use the leatest version.

Query interface

In [4]:
params={'output.fmt':'csv','obsidTextarea':'353301001'}
hl=lm.query(params)
showHtml(hl)

In [5]:
params={'output.fmt':'csv','pos.type':'proximity'}
files={'pos.posfile':('sample.txt', open('sample.txt', 'r'))}
hl=lm.query2(params, files)
showHtml(hl)

SQL query interface

In [6]:
lm.sql('select 1')

'#\tPowered by Chinese Virtual Observatory (China-VO): www.china-vo.org 2018-01-29 15:31:46\n#\tIn case of problem: please contact support@lamost.org\n#----------------------------------------------------------------------------\n#\t?column?|null|null|null\n#----------------------------------------------------------------------------\n?column?\n1\n'

simple information

In [7]:
lm.getInfo('353301001')

'{"response":[{"what":"obsid","action":"","data":"353301001"},{"what":"designation","action":"","data":"J224415.49+193359.7"},{"what":"obsdate","action":"","data":"2015-09-12"},{"what":"mjd","action":"","data":"57277"},{"what":"lmjd","action":"","data":"57278"},{"what":"planid","action":"","data":"EG224429N215706B01"},{"what":"spid","action":"","data":"1"},{"what":"fiberid","action":"","data":"1"},{"what":"ra","action":"","data":"341.0645660000"},{"what":"dec","action":"","data":"19.5665910000"},{"what":"class","action":"","data":"STAR"},{"what":"subclass","action":"","data":"K7"},{"what":"z","action":"","data":"0.0000400277"},{"what":"logg","action":"","data":"4.701"},{"what":"teff","action":"","data":"4141.41"},{"what":"feh","action":"","data":"-0.323"},{"what":"rv","action":"","data":"12.00"}]}'

download FITS file

In [8]:
lm.downloadFits(obsid='353301001',savedir='./')

download png

In [9]:
lm.downloadPng(obsid='353301007',savedir='./')

download csv format spectrum

In [10]:
csv = lm.getFitsCsv(obsid='353301007')
#print(csv)

Cone Search Protocol

In [11]:
cs = lm.conesearch(ra=10.0004738,dec=40.9952444,radius=0.2)
#print(cs)

Simple Spectral Access Protocol

In [12]:
ssap = lm.ssap(ra=10.0004738,dec=40.9952444,radius=0.2)
#print(ssap)

read local spectrum fits to data array, then same result as getFitsCsv() which is online function

In [13]:
wavelength, specflux, spec_smooth_7, spec_smooth_15=lm.readFits('spec-57278-EG224429N215706B01_sp01-001.fits.gz')
#print('wavelength', wavelength)
#print('specflux', specflux)
#print('spec_smooth_7', spec_smooth_7)
#print('spec_smooth_15', spec_smooth_15)

C:\Anaconda3-v5.0.1-x64\lib\site-packages\pyfits-3.5-py3.6-win-amd64.egg\pyfits\util.py:899: UserWarning: Could not find appropriate MS Visual C Runtime library or library is corrupt/misconfigured; cannot determine whether your file object was opened in append mode.  Please consider using a file object opened in write mode instead.
  'Could not find appropriate MS Visual C Runtime '
